In [9]:
"""
Author: Sebastian Vendt, University of Ulm

This is part of the implementation of the four different neural networks proposed in the paper 
Recurrent Convolutional Neural Networks: A Better Model of Biological Object Recognition
of Courtney J. Spoerer et al. 

This script implements the bottom up convolutional network including lateral connections in the two hidden layers

"""

include("./dataManager.jl")
using Flux, Statistics
using Flux: onehot, onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
import LinearAlgebra: norm
using NNlib
using FeedbackConvNets

using MAT # needs installation of Pkg.add("MAT")
using PyPlot # pip install Matplotlib Pkg.add("PyPlot")
using ..dataManager: make_batch
using Base

using ProgressMeter
using Printf
using Test
using Pkg

norm(x::TrackedArray{T}) where T = sqrt(sum(abs2.(x)) + eps(T)) 


# PARAMETERS
const batch_size = 100
const n = 5
const c = 1
const alpha = Float32(10^-4)
const beta = 0.5f0
const lambda = 0.0005f0

const kernelsize = (3, 3)
const featuresize = 32

const padding = (Int64(floor(kernelsize[1]/2)), Int64(floor(kernelsize[2]/2)))
const momentum = 0.9f0
init_learning_rate = 0.01f0
learning_rate = init_learning_rate
const epochs = 100
const decay_rate = 0.1f0
const decay_step = 40
const fc_output_units = 10
const input_size=(32,32) # Width x Height
# end of PARAMETERS

train_idx = 1
epoch_idx = 1

@printf("Constructing model...\n")
chain = FeedbackChain(
    # HIDDEN LAYER 1
    # Input Image 32x32x1xN
    Merger("lateral_1", Conv(kernelsize, featuresize=>1, pad=padding, identity), +),
    Conv(kernelsize, 1=>featuresize, pad=padding, relu),
    x -> localResponseNorm(x),
    Forker("lateral_1"),
    
    MaxPool((2, 2), stride=(2, 2)),
    
    # HIDDEN LAYER 2
    # Input Image 16x16x32xN
    Merger("lateral_2", Conv(kernelsize, featuresize=>featuresize, pad=padding, identity), +),
    Conv(kernelsize, featuresize=>featuresize, pad=padding, relu),
    x -> localResponseNorm(x),
    Forker("lateral_2"),
    MaxPool((Int64(input_size[1] / 2), Int64(input_size[2] / 2)), stride=(1, 1)),
    # reshape to 32xN
    x -> reshape(x, :, size(x, 4)),
    Dense(featuresize, fc_output_units, σ),
)





Constructing model...


FeedbackChain{Tuple{Merger{Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},typeof(+)},Conv{2,2,typeof(relu),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},getfield(Main, Symbol("##51#54")),Forker,MaxPool{2,4},Merger{Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},typeof(+)},Conv{2,2,typeof(relu),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},getfield(Main, Symbol("##52#55")),Forker,MaxPool{2,4},getfield(Main, Symbol("##53#56")),Dense{typeof(σ),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}}}((Merger{Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},typeof(+)}("lateral_1", Conv((3, 3), 32=>1), +), Conv((3, 3), 1=>32, NNlib.relu), getfield(Main, Symbol("##51#54"))(), Forker("lateral_1"), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Merger{Conv{2,2,typeof(identity),TrackedArray{…,Array{Floa